In [ ]:
# 新しいrelative strengthを基準としたwatch listを作成する
import datetime
from pathlib import Path

import numpy as np
import polars as pl
import matplotlib.pyplot as plt

import stock

In [ ]:
csv_dir = stock.PROJECT_ROOT / "data/daily"

In [ ]:
def get_relative_strength_codes(target_date: datetime.date, csv_list: list[Path] | None = None) -> list[Path]:
    # relative strengthが100を超えた銘柄を取得
    target_list = []
    csv_list = sorted(csv_dir.glob("*.csv")) if csv_list is None else csv_list
    for csv_path in csv_list:
        df = stock.kabutan.read_data_csv(csv_path, end_date=target_date)
        if len(df) < 2:
            continue
        target_rs, prev_rs = df["rs"][-1], df["rs"][-2]
        if target_rs < 0 or prev_rs < 0:
            continue
        if prev_rs < 1.0 and 1.0 < target_rs:
            target_list.append(csv_path)
    return target_list

In [ ]:
def get_near_high_codes(target_date: datetime.date, csv_list: list[Path] | None = None) -> list[Path]:
    # 過去の高値（52高値？）付近に株価
    min_rate_from_low = 0.2  # 安値から何割以上高くなっているか
    near_from_high = 0.2  # 高値からどれくらい近くにいるか
    target_list = []
    csv_list = sorted(csv_dir.glob("*.csv")) if csv_list is None else csv_list
    for csv_path in csv_list:
        df = stock.kabutan.read_data_csv(csv_path, start_date=target_date - datetime.timedelta(days=365), end_date=target_date)
        min_val = df["low"].min()
        max_val = df["high"].max()
        cur_val = df["close"][-1]
        if cur_val > min_val * (1 + min_rate_from_low) and cur_val > max_val * (1 - near_from_high):
            target_list.append(csv_path)
    return target_list

In [ ]:
def get_uptrend_codes(target_date: datetime.date, csv_list: list[Path] | None = None) -> list[Path]:
    csv_list = sorted(csv_dir.glob("*.csv")) if csv_list is None else csv_list
    target_list = []
    higher_price_weeks = [10, 30, 40]
    up_trend_weeks = [40]
    for csv_path in csv_list:
        df = stock.kabutan.read_data_csv(csv_path, end_date=target_date)
        
        flag = True
        # 移動平均線より株価が高いか
        avgs = stock.trend_template.technical.calc_mean_average(df, weeks=higher_price_weeks, cur_day=target_date, target_days=1)
        flag &= all([avg[0] < df["close"][-1] for avg in avgs])
        # 移動平均線が上昇トレンドかチェック
        avgs = stock.trend_template.technical.calc_mean_average(df, weeks=up_trend_weeks, cur_day=target_date, target_days=10)
        flag &= all([avg[0] < avg[-1] for avg in avgs])
            
        if flag:
            target_list.append(csv_path)
    return target_list

In [ ]:
target_date = datetime.date(year=2024, month=4, day=1)

target_list = get_relative_strength_codes(target_date)
target_list = get_near_high_codes(target_date, target_list)
target_list = get_uptrend_codes(target_date, target_list)

In [ ]:
len(target_list)

In [ ]:
def plot_with_rs(df, start_date, end_date=datetime.datetime.today()):
    fig = plt.figure()
    df = df.filter((pl.col("date") >= start_date) & (pl.col("date") <= end_date))
    ax1 = fig.add_subplot(111)
    ax1.plot(df["date"], df["close"], "red", label="close")

    ax2 = ax1.twinx()
    ax2.plot(df["date"], df["rs"], "blue", label="rs")
    ax2.grid("both")
    ax2.tick_params(axis='x', labelrotation=90)

    plt.legend()

In [ ]:
def calc_rs(dates, df, ref_df, days=30, use_weight=True):
    min_date = df["date"].min()
    res = -np.ones(len(dates))
    for idx, date in enumerate(sorted(dates)):
        start_date = date - datetime.timedelta(days=days)
        if start_date < min_date:
            continue
        res[idx] = stock.relative_strength.relative_strength_v2(
            df, ref_df, start_date=start_date, end_date=date, use_weight=use_weight)
    return pl.Series(res)

In [ ]:
code = "6254"
csv_path = csv_dir / f"{code}.csv"
df = stock.kabutan.read_data_csv(csv_path)
ref_df = stock.kabutan.read_data_csv(csv_dir / "0010.csv")
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365)

In [ ]:
df = df.with_columns(pl.col("date").map_batches(lambda val : calc_rs(val, df, ref_df, use_weight=False)).alias("rs"))
plot_with_rs(df, start_date=start_date, end_date=end_date)

In [ ]:
df = df.with_columns(pl.col("date").map_batches(lambda val : calc_rs(val, df, ref_df, use_weight=True)).alias("rs"))
plot_with_rs(df, start_date=start_date, end_date=end_date)